# read the input
- first line: N (số thành phố cần thăm) (đi từ thành phố 0 đến thành phố N-1)
- ma trận khoảng cách d(i,j): khoảng cách giữa thành phố i & j

In [4]:
from ortools.linear_solver import pywraplp


def read_input():
    # N
    N = int(input())
    # Read distance matrix
    distance_matrix = []
    for _ in range(0,N):
        row = list(map(int, input().split()))
        distance_matrix.append(row)
        
    return N, distance_matrix



# create decision vars
- X[i,j]: binary var, if there exist direct path from i to j


In [5]:
def create_variables(solver, N):
   
    X = {}
    for i in range(0,N):
        for j in range(0,N):
            X[i, j] = solver.BoolVar(f'X[{i},{j}]')
    
    return X

# add constraints
- Vào 1 lần ra 1 lần :  sum of X[i,j] (j from 0 to N-1) = sum of X[j,i] (j from 0 to N-1) = 1 
- Subtour Elimination: 

In [17]:
def add_constraints(solver, N,X, distance_matrix):
    
    for i in range(N):
        solver.Add(solver.Sum(X[i, j] for j in range(N) if i != j) == 1)  # Exit constraint
        solver.Add(solver.Sum(X[j, i] for j in range(N) if i != j) == 1)  # Enter constraint
    # MTZ Constraints for Subtour Elimination
    # U la thu tu tham cac thanh pho
    
    u = [solver.IntVar(0, N - 1, f'u[{i}]') for i in range(N)]
    
    # 
    for i in range(1, N):
        for j in range(1, N):
            if i != j:
                solver.Add(u[i] - u[j] + N * X[i, j] <= N - 1)
    

# extract route


In [7]:
def extract_routes(X,N):
    solution = []
    for i in range(N):
        for j in range(N):
            if i != j and X[i, j].solution_value() > 0.5:
                solution.append((i, j))
    return solution

# main

In [22]:
def main():
    # Create solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    
    # Read input
    N = 4
    distance_matrix= [[0, 10, 15, 20], [10, 0, 35, 25], [15, 35, 0, 30], [20, 25, 30, 0]] 
    
    # Create variables
    X = create_variables(solver, N)
    
    # Set objective: minimize maximum distance
    s = sum(distance_matrix[i][j] * X[i, j] for i in range(N) for j in range(N) if i != j)
    solver.Minimize(s)
    
    # Add constraints
    add_constraints(solver, N,X, distance_matrix)
    
    # Solve the problem
    status = solver.Solve()
    
    # Output results
    if status == pywraplp.Solver.OPTIMAL:
        print(s.solution_value())
        routes = extract_routes(X,N)
        print(routes)
    else:
        print('No optimal solution found.')

main()

80.0
[(0, 1), (1, 3), (2, 0), (3, 2)]
